In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import matplotlib as plt
import operator

In [2]:
df = pd.read_csv('Reviews.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
Id                        568454 non-null int64
ProductId                 568454 non-null object
UserId                    568454 non-null object
ProfileName               568438 non-null object
HelpfulnessNumerator      568454 non-null int64
HelpfulnessDenominator    568454 non-null int64
Score                     568454 non-null int64
Time                      568454 non-null int64
Summary                   568427 non-null object
Text                      568454 non-null object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [ ]:
def data_clean(df, feature, x):
    count = df[feature].value_counts()
    df = df[df[feature].isin(count[count > x].index)]
    return df
def arrange_data(df,features,x):
    fil = df.ProductId.value_counts()
    fil2 = df.UserId.value_counts()
    df['nProducts'] = df.ProductId.apply(lambda n: fil[n])
    df['nUsers'] = df.UserId.apply(lambda n: fil2[n])
    while (df.ProductId.value_counts(ascending=True)[0]) < x or  (df.UserId.value_counts(ascending=True)[0] < x):
        df = data_clean(df,features[0],x)
        df = data_clean(df,features[1],x)
    return df
processed_data_frame = arrange_data(df, ['ProductId', 'UserId'], 10)

In [ ]:
processed_data_frame.head()

In [ ]:
processed_data_frame['uid'] = pd.factorize(processed_data_frame['UserId'])[0]
processed_data_frame['pid'] = pd.factorize(processed_data_frame['ProductId'])[0]

In [ ]:
processed_data_frame.head()

In [ ]:
processed_data_frame.hist(bins = 50, figsize = (20,15))

In [ ]:
processed_data_frame.info()

In [ ]:
X = processed_data_frame.loc[:,['uid','pid','Time','nUsers','nProducts']]
Y = processed_data_frame.Score
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=2017)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(X_train,Y_train)
Y_test_predict = model.predict(X_test)

In [ ]:
temp1 = mean_squared_error(Y_test,Y_test_predict)
print('Mean square error on testing data: ',temp1)


recommendation_accuracy = Y_test_predict >= 4
temp1 = Y_test >= 4

no_recommendation_accuracy = Y_test_predict < 3
temp2 = Y_test < 3

print('Accuracy score of recommendation: ',accuracy_score(temp1,recommendation_accuracy))
print('Accuracy score of not recommendation: ',accuracy_score(temp2,no_recommendation_accuracy))


In [ ]:
prid = processed_data_frame.ProductId
score = processed_data_frame.Score
def recomned_product(data_framee,prid_array,score_array,score_predicted):
    pid_score = dict()
    for i in range(len(prid_array)):
        if score_array[i] >= score_predicted:
            if prid_array[i] in pid_score.keys() and score_array[i] >= score_predicted:
                pid_score[prid_array[i]] = pid_score[prid_array[i]]+1
            else:
                pid_score[prid_array[i]] = 0
    pid_score_sorted = {}
    pid_score_sorted = dict(sorted(pid_score.items(), key=operator.itemgetter(1),reverse = True))
    type(pid_score_sorted)
    print("recommended product for given data of user")
    c = 0
    product = []
    for i in pid_score_sorted:
        product.append(i)
        c = c+1
        if c == 10:
            return product;
    
data = [[124,20,0.899981,0.099515,0.999667]]
data_framee = pd.DataFrame(data, columns = ['uid','pid','time','nuser','nproduct'])
score_predicted = model.predict(data_framee)
prid_array = pd.Series(prid).array
score_array = pd.Series(score).array

product = recomned_product(data_framee,prid_array,score_array,score_predicted)
print(product)